### System Installation
Installing the coref tool from mandarjoshi90 along with tensorflow.




In [1]:
!nvidia-smi --query-gpu=gpu_name,driver_version,memory.total --format=csv

name, driver_version, memory.total [MiB]
Tesla K80, 460.32.03, 11441 MiB


In [2]:
! git clone https://github.com/mandarjoshi90/coref.git

Cloning into 'coref'...
remote: Enumerating objects: 734, done.
remote: Counting objects: 100% (6/6), done.
remote: Compressing objects: 100% (6/6), done.
remote: Total 734 (delta 2), reused 0 (delta 0), pack-reused 728
Receiving objects: 100% (734/734), 4.17 MiB | 11.31 MiB/s, done.
Resolving deltas: 100% (441/441), done.


In [3]:
%cd coref
! sed 's/MarkupSafe==1.0/MarkupSafe==1.1.1/; s/scikit-learn==0.19.1/scikit-learn==0.21/; s/scipy==1.0.0/scipy==1.6.2/' < requirements.txt > tmp
! mv tmp requirements.txt

! sed 's/.D.GLIBCXX.USE.CXX11.ABI.0//' < setup_all.sh  > tmp
! mv tmp setup_all.sh 
! chmod u+x setup_all.sh 

/content/coref


In [4]:
%tensorflow_version 2.x
! pip uninstall -y tensorflow
! pip install -r requirements.txt --log install-log.txt -q
! ./setup_all.sh

Uninstalling tensorflow-2.4.1:
  Successfully uninstalled tensorflow-2.4.1
     |████████████████████████████████| 102kB 4.2MB/s 
     |████████████████████████████████| 1.2MB 28.7MB/s 
     |████████████████████████████████| 163kB 36.6MB/s 
     |████████████████████████████████| 6.6MB 20.5MB/s 
     |████████████████████████████████| 552kB 22.3MB/s 
     |████████████████████████████████| 61kB 7.0MB/s 
     |████████████████████████████████| 2.2MB 38.3MB/s 
     |████████████████████████████████| 266kB 37.2MB/s 
     |████████████████████████████████| 890kB 31.6MB/s 
     |████████████████████████████████| 133kB 43.9MB/s 
     |████████████████████████████████| 153kB 38.9MB/s 
     |████████████████████████████████| 51kB 6.3MB/s 
     |████████████████████████████████| 51kB 5.7MB/s 
     |████████████████████████████████| 92kB 9.9MB/s 
     |████████████████████████████████| 20.3MB 2.0MB/s 
     |████████████████████████████████| 2.1MB 14.2MB/s 
     |████████████████████████████████

In [5]:
import nltk
nltk.download('punkt')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


True

### Specifying Input

Input and Model

In [6]:
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [7]:
genre = "nw"
# The Ontonotes data for training the model contains text from several sources
# of very different styles. You need to specify the most suitable one out of:
# "bc": broadcast conversation
# "bn": broadcast news
# "mz": magazine
# "nw": newswire
# "pt": Bible text
# "tc": telephone conversation
# "wb": web data

model_name = "spanbert_base"
# The fine-tuned model to use. Options are:
# bert_base
# spanbert_base
# bert_large
# spanbert_large

In [22]:
import os
os.environ['data_dir'] = "./data"
os.environ['CHOSEN_MODEL'] = model_name

Downloading the selected model.

In [20]:
! ./download_pretrained.sh $CHOSEN_MODEL

--2021-05-17 12:46:11--  http://nlp.cs.washington.edu/pair2vec/spanbert_base.tar.gz
Resolving nlp.cs.washington.edu (nlp.cs.washington.edu)... 128.208.3.120, 2607:4000:200:12::78
Connecting to nlp.cs.washington.edu (nlp.cs.washington.edu)|128.208.3.120|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1633726311 (1.5G) [application/x-gzip]
/content/gdrive/CRAFT-conll/train: No such file or directory
/content/gdrive/CRAFT-conll/train/spanbert_base.tar.gz: No such file or directory

Cannot write to ‘/content/gdrive/CRAFT-conll/train/spanbert_base.tar.gz’ (No such file or directory).
tar (child): /content/gdrive/CRAFT-conll/train//spanbert_base.tar.gz: Cannot open: No such file or directory
tar (child): Error is not recoverable: exiting now
tar: Child returned status 2
tar: Error is not recoverable: exiting now
rm: cannot remove '/content/gdrive/CRAFT-conll/train//spanbert_base.tar.gz': No such file or directory


Process the data to be in the required input format:

In [10]:
from bert import tokenization
import json


def tokenize(data):
  data = {
      'doc_key': genre,
      'sentences': [["[CLS]"]],
      'speakers': [["[SPL]"]],
      'clusters': [],
      'sentence_map': [0],
      'subtoken_map': [0],
  }

  # Determine Max Segment
  max_segment = None
  for line in open('experiments.conf'):
      if line.startswith(model_name):
          max_segment = True
      elif line.strip().startswith("max_segment_len"):
          if max_segment:
              max_segment = int(line.strip().split()[-1])
              break

  tokenizer = tokenization.FullTokenizer(vocab_file="cased_config_vocab/vocab.txt", do_lower_case=False)
  subtoken_num = 0
  for sent_num, line in enumerate(text):
      raw_tokens = line.split()
      tokens = tokenizer.tokenize(line)
      if len(tokens) + len(data['sentences'][-1]) >= max_segment:
          data['sentences'][-1].append("[SEP]")
          data['sentences'].append(["[CLS]"])
          data['speakers'][-1].append("[SPL]")
          data['speakers'].append(["[SPL]"])
          data['sentence_map'].append(sent_num - 1)
          data['subtoken_map'].append(subtoken_num - 1)
          data['sentence_map'].append(sent_num)
          data['subtoken_map'].append(subtoken_num)

      ctoken = raw_tokens[0]
      cpos = 0
      for token in tokens:
          data['sentences'][-1].append(token)
          data['speakers'][-1].append("-")
          data['sentence_map'].append(sent_num)
          data['subtoken_map'].append(subtoken_num)
          
          if token.startswith("##"):
              token = token[2:]
          if len(ctoken) == len(token):
              subtoken_num += 1
              cpos += 1
              if cpos < len(raw_tokens):
                  ctoken = raw_tokens[cpos]
          else:
              ctoken = ctoken[len(token):]

  data['sentences'][-1].append("[SEP]")
  data['speakers'][-1].append("[SPL]")
  data['sentence_map'].append(sent_num - 1)
  data['subtoken_map'].append(subtoken_num - 1)

  return data

def chunks(lst, n):
    """Yield successive n-sized chunks from lst."""
    for i in range(0, len(lst), n):
        yield lst[i:i + n]

/usr/local/lib/python3.7/dist-packages/tensorflow/python/framework/dtypes.py:516: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/usr/local/lib/python3.7/dist-packages/tensorflow/python/framework/dtypes.py:517: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
/usr/local/lib/python3.7/dist-packages/tensorflow/python/framework/dtypes.py:518: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
/usr/local/lib/python3.7/dist-packages/tensorflow/python/framework/dtypes.py:519: FutureWarning: Passing (type, 1) or 

In [ ]:
%mkdir ./data/training

In [23]:
os.environ['train_dir'] = "/content/gdrive/MyDrive/CRAFT-conll/train/"

In [40]:
import shutil

src = "/content/gdrive/MyDrive/CRAFT-conll/train/11532192.conll"
dst = "./data/dev.english.v4_gold_conll"
shutil.copy(src,dst)

'./data/dev.english.v4_gold_conll'

In [ ]:
! ./setup_training.sh $train_dir $data_dir

In [44]:
! python minimize.py ./data/cased_L-24_H-1024_A-16/vocab.txt ./data/ ./data/ true

/usr/local/lib/python3.7/dist-packages/tensorflow/python/framework/dtypes.py:516: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/usr/local/lib/python3.7/dist-packages/tensorflow/python/framework/dtypes.py:517: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
/usr/local/lib/python3.7/dist-packages/tensorflow/python/framework/dtypes.py:518: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
/usr/local/lib/python3.7/dist-packages/tensorflow/python/framework/dtypes.py:519: FutureWarning: Passing (type, 1) or 

In [48]:
! GPU=0 python train.py train_spanbert_base

/usr/local/lib/python3.7/dist-packages/tensorflow/python/framework/dtypes.py:516: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/usr/local/lib/python3.7/dist-packages/tensorflow/python/framework/dtypes.py:517: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
/usr/local/lib/python3.7/dist-packages/tensorflow/python/framework/dtypes.py:518: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
/usr/local/lib/python3.7/dist-packages/tensorflow/python/framework/dtypes.py:519: FutureWarning: Passing (type, 1) or 

## Prediction & Evaluation


In [11]:
%mkdir ./data/in
%mkdir ./data/out

In [49]:
import subprocess
tokenizer = nltk.data.load('tokenizers/punkt/english.pickle')

filename = "/content/gdrive/MyDrive/CRAFT-txt/dev/17194222.txt"
text = []
data = []

with open(filename) as f_in:
  
  paragraphs = [line for line in f_in.read().split('\n') if line]

  for paragraph in paragraphs:
    sentences = tokenizer.tokenize(paragraph)
    text.extend(sentences)
              
chs = list(chunks(text, 100))

N = len(chs)

for i in range(0, N):
  
  text = chs[i]
  data.append(tokenize(text))

  file1 = "./data/in/"  + "test" + "_" + str(i) + ".json"
  file2 = "./data/out/" + "test" + "_" + str(i) + ".jsonlines"

  with open(file1, 'w') as out:
    json.dump(data[i], out, sort_keys=True)
  
  subprocess.call(['python', 'predict.py', 'spanbert_base', file1, file2])


KeyboardInterrupt: ignored

In [13]:
import subprocess
tokenizer = nltk.data.load('tokenizers/punkt/english.pickle')

for file in [f for f in os.listdir('/content/gdrive/MyDrive/CRAFT-txt/test') if f.endswith('.txt')]:

  text = []
  data = []

  with open(filename) as f_in:
    
    paragraphs = [line for line in f_in.read().split('\n') if line]
  
    for paragraph in paragraphs:
      sentences = tokenizer.tokenize(paragraph)
      text.extend(sentences)
                
  chs = list(chunks(text, 100))

  N = len(chs)

  for i in range(0, N):
    
    text = chs[i]
    data.append(tokenize(text))

    file1 = "./data/in/"  + file[0:-4] + "_" + str(i) + ".json"
    file2 = "./data/out/" + file[0:-4] + "_" + str(i) + ".jsonlines"

    with open(file1, 'w') as out:
      json.dump(data[i], out, sort_keys=True)
    
    subprocess.call(['python', 'predict.py', 'spanbert_base', file1, file2])


In [50]:
! gpu=0 python predict.py train_spanbert_base ./data/in/test_0.json ./data/out/test_0.json

/usr/local/lib/python3.7/dist-packages/tensorflow/python/framework/dtypes.py:516: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/usr/local/lib/python3.7/dist-packages/tensorflow/python/framework/dtypes.py:517: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
/usr/local/lib/python3.7/dist-packages/tensorflow/python/framework/dtypes.py:518: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
/usr/local/lib/python3.7/dist-packages/tensorflow/python/framework/dtypes.py:519: FutureWarning: Passing (type, 1) or 

### Using Evaulate
currently not working

In [ ]:
! gpu=0 python evaluate.py $CHOSEN_MODEL

## Output

In [52]:
output = json.load(open("./data/out/test_0.json"))

comb_text = [word for sentence in output['sentences'] for word in sentence]

def convert_mention(mention):
    start = output['subtoken_map'][mention[0]]
    end = output['subtoken_map'][mention[1]] + 1
    nmention = (start, end)
    mtext = ''.join(' '.join(comb_text[mention[0]:mention[1]+1]).split(" ##"))
    return (nmention, mtext)

seen = set()
print('Clusters:')
for cluster in output['predicted_clusters']:
    mapped = []
    for mention in cluster:
        seen.add(tuple(mention))
        mapped.append(convert_mention(mention))
    print(mapped, end=",\n")

print('\nMentions:')
for mention in output['top_spans']:
    if tuple(mention) in seen:
        continue
    print(convert_mention(mention), end=",\n")

Clusters:

Mentions:
((0, 1), 'Gene'),
((0, 1), 'Genetic'),
((0, 15), 'Genetic Analysis of the Roles of BMP2 , BMP4 , and BMP7 in Limb Patterning and Skeleto'),
((6, 7), 'BMP'),
((7, 8), 'B'),
((7, 8), 'BMP'),
((9, 10), 'BMP'),
((9, 10), '##MP'),
((11, 12), 'Limb'),
((12, 13), 'Pat'),
((12, 13), 'Pattern'),
((14, 15), '##keleto'),
((15, 16), 'Abs'),
((17, 18), 'mor'),
((17, 18), '##or'),
((17, 18), '##phogen'),
((19, 20), 'B'),
((19, 20), 'BMP'),
((19, 24), 'BMP ) family members , including BMP2'),
((23, 24), '##MP2'),
((24, 25), 'BMP'),
((26, 27), 'B'),
((26, 27), 'BMP'),
((26, 27), '##MP'),
((30, 32), 'limb development'),
((31, 32), 'development'),
((32, 33), 'B'),
((32, 33), 'BMP'),
((32, 48), 'BMPs have been implicated in early limb patterning as well as in the process of skeleto'),
((38, 39), 'limb'),
((39, 40), 'pattern'),
((39, 40), 'patterning'),
((47, 48), '##keleto'),
((47, 48), '##eto'),
((48, 49), 'However ,'),
((55, 56), 'emb'),
((55, 56), '##b'),
((55, 56), '##ryo'),
((56

In [14]:
!zip -r /content/out.zip /content/coref/data/out 

  adding: content/coref/data/out/ (stored 0%)
  adding: content/coref/data/out/17465682_3.jsonlines (deflated 84%)
  adding: content/coref/data/out/16611361_3.jsonlines (deflated 84%)
  adding: content/coref/data/out/15850489_3.jsonlines (deflated 84%)
  adding: content/coref/data/out/16787536_3.jsonlines (deflated 84%)
  adding: content/coref/data/out/16517939_3.jsonlines (deflated 84%)
  adding: content/coref/data/out/15615595_0.jsonlines (deflated 82%)
  adding: content/coref/data/out/15238161_1.jsonlines (stored 0%)
  adding: content/coref/data/out/15238161_0.jsonlines (deflated 82%)
  adding: content/coref/data/out/16026622_0.jsonlines (deflated 82%)
  adding: content/coref/data/out/15018652_4.jsonlines (deflated 85%)
  adding: content/coref/data/out/17503968_3.jsonlines (deflated 84%)
  adding: content/coref/data/out/15615595_4.jsonlines (deflated 85%)
  adding: content/coref/data/out/16026622_3.jsonlines (deflated 84%)
  adding: content/coref/data/out/17206865_1.jsonlines (defla

In [16]:
from google.colab import files
files.download("/content/out.zip")

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>